In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
TR = 12e-3  # Repetition Time in seconds
TE = 3.15e-3  # Echo Time in seconds
flip_angle_amplitude = 0.25  # Scaled flip angle amplitude for RF

# Time vector for two TR cycles
t_full = np.linspace(0, 2 * TR, 1000)

# Generate sinc RF pulse
def sinc_rf(t, start, end):
    sinc_wave = np.sinc(np.linspace(-4, 4, len(t[(t >= start) & (t < end)])))
    rf_wave = np.zeros_like(t)
    rf_wave[(t >= start) & (t < end)] = sinc_wave
    return rf_wave

RF_full_sinc_rescaled = sinc_rf(t_full, 0, TE) * flip_angle_amplitude + sinc_rf(t_full, TR, TR + TE) * flip_angle_amplitude

# Generate G_slice gradient
def Gslice(t, start, end, positive_amplitude=0.25, negative_amplitude=-0.25, positive_width_ratio=2):
    gradient_wave = np.zeros_like(t)
    total_width = end - start
    negative_width = total_width / (2 + positive_width_ratio)
    positive_width = negative_width * positive_width_ratio
    gradient_wave[(t >= start) & (t < start + negative_width)] = negative_amplitude
    gradient_wave[(t >= start + negative_width) & (t < start + negative_width + positive_width)] = positive_amplitude
    gradient_wave[(t >= start + negative_width + positive_width) & (t < end)] = negative_amplitude
    return gradient_wave

G_slice_adjusted_amplitude = Gslice(t_full, 0, TE) + Gslice(t_full, TR, TR + TE)

# Generate G_phase gradient
def Gphase(t, start, end, amplitude=0.25):
    gradient_wave = np.zeros_like(t)
    total_width = end - start
    negative_width = total_width / (2 + 2)
    positive_width = negative_width * 2
    gradient_wave[(t >= start) & (t < start + negative_width)] = -amplitude
    gradient_wave[(t >= start + negative_width + positive_width) & (t < end)] = amplitude

    dotted_lobes = np.zeros_like(gradient_wave)
    dotted_lobes[gradient_wave < 0] = -gradient_wave[gradient_wave < 0]
    dotted_lobes[gradient_wave > 0] = -gradient_wave[gradient_wave > 0]

    return gradient_wave, dotted_lobes

# Compute G_phase and its dotted lobes separately for each time range
G_phase_1, G_phase_dotted_1 = Gphase(t_full, 0, TE)
G_phase_2, G_phase_dotted_2 = Gphase(t_full, TR, TR + TE)
G_phase_matching = G_phase_1 + G_phase_2
G_phase_dotted = G_phase_dotted_1 + G_phase_dotted_2

# Generate G_readout gradient
def Greadout(t, center, positive_width, G_phase, amplitude=0.25, center_amplitude=0.25):
    readout_wave = np.zeros_like(t)
    center_start = center - positive_width / 2
    center_end = center + positive_width / 2
    readout_wave[(t >= center_start) & (t < center_end)] = center_amplitude

    readout_wave[G_phase != 0] = -amplitude
    return readout_wave

positive_lobe_width = TE / (2 + 2) * 2
G_readout_with_negative_lobes = Greadout(
    t_full, TE/2 + TR / 2, positive_width=positive_lobe_width, G_phase=G_phase_matching
)

plt.figure(figsize=(12, 10))

# RF plot
plt.subplot(5, 1, 1)
plt.plot(t_full * 1e3, RF_full_sinc_rescaled, color="b")
plt.ylabel("RF", fontsize = 20)
plt.xticks([])
plt.grid(False)

# G_slice plot
plt.subplot(5, 1, 2)
plt.plot(t_full * 1e3, G_slice_adjusted_amplitude, color="r")
plt.ylabel("G Slice", fontsize = 20)
plt.xticks([])
plt.grid(False)

# G_phase plot with dotted complementary lobes
plt.subplot(5, 1, 3)
plt.plot(t_full * 1e3, G_phase_matching, color="g")
plt.plot(t_full * 1e3, G_phase_dotted, linestyle="--", color="g", alpha=0.7)
plt.ylabel("G Phase", fontsize = 20)
plt.xticks([])
plt.grid(False)

# G_readout plot
plt.subplot(5, 1, 4)
plt.plot(t_full * 1e3, G_readout_with_negative_lobes, color="m")
plt.ylabel("G Readout", fontsize = 20)
plt.xlabel("Time (ms)", fontsize = 20)
plt.grid(False)

plt.tight_layout()
plt.show()